In [6]:
import ccxt
import time
import pandas as pd
import talib
import numpy as np
import matplotlib.pyplot as plt
import os
import uuid
from scipy.signal import argrelextrema
from datetime import datetime, timedelta
import bisect
#Performence:
import logging
import math
# Konfiguracja loggera
logging.basicConfig(
    filename='logs.txt',        # Nazwa pliku logów
    level=logging.INFO,         # Poziom logowania
    format='%(asctime)s - %(levelname)s - %(message)s'  # Format logów
)

interval_mapping = {
    '1M': {'name': '1M', 'dataOffset': pd.Timedelta(days=31) , 'minimum_decreasing_count': 7  ,'minimum_decreasing_count': 6, 'max_different_atr_from_MACD': 0.08, 'max_ATR_to_for_consolidation': 0.1 , 'min_time_for_consolidation': 3, },
    '2w': {'name': '1w', 'dataOffset': pd.Timedelta(days=14) , 'minimum_increasing_count': 7 ,'minimum_decreasing_count': 6 , 'max_different_atr_from_MACD': 0.08, 'max_ATR_to_for_consolidation': 0.15 , 'min_time_for_consolidation': 3},
    '1w': {'name': '1w', 'dataOffset': pd.Timedelta(days=7) , 'minimum_increasing_count': 7 ,'minimum_decreasing_count': 6, 'max_different_atr_from_MACD': 0.08, 'max_ATR_to_for_consolidation': 0.25 , 'min_time_for_consolidation': 3},
    '3d': {'name': '3d', 'dataOffset': pd.Timedelta(days=3), 'minimum_increasing_count': 7 ,'minimum_decreasing_count': 6, 'max_different_atr_from_MACD': 0.12, 'max_ATR_to_for_consolidation': 0.25 , 'min_time_for_consolidation': 3},
    '2d': {'name': '2d', 'dataOffset': pd.Timedelta(days=2), 'minimum_increasing_count': 7 ,'minimum_decreasing_count': 6, 'max_different_atr_from_MACD': 0.12, 'max_ATR_to_for_consolidation': 0.2 , 'min_time_for_consolidation': 3},
    '1d': {'name': '1d', 'dataOffset': pd.Timedelta(days=1), 'minimum_increasing_count': 7 ,'minimum_decreasing_count': 6, 'max_different_atr_from_MACD': 0.15, 'max_ATR_to_for_consolidation': 0.2 , 'min_time_for_consolidation': 3},
    '12h': {'name': '12h', 'dataOffset': pd.Timedelta(hours=12), 'minimum_increasing_count': 8 ,'minimum_decreasing_count': 6, 'max_different_atr_from_MACD': 0.18, 'max_ATR_to_for_consolidation': 0.3 , 'min_time_for_consolidation': 3},
    '8h': {'name': '8h', 'dataOffset': pd.Timedelta(hours=8), 'minimum_increasing_count': 8 ,'minimum_decreasing_count': 6, 'max_different_atr_from_MACD': 0.2, 'max_ATR_to_for_consolidation': 0.3 , 'min_time_for_consolidation': 4},
    '7h': {'name': '7h', 'dataOffset': pd.Timedelta(hours=7), 'minimum_increasing_count': 8 ,'minimum_decreasing_count': 6, 'max_different_atr_from_MACD': 0.2, 'max_ATR_to_for_consolidation': 0.3 , 'min_time_for_consolidation': 4},
    '6h': {'name': '6h', 'dataOffset': pd.Timedelta(hours=6), 'minimum_increasing_count': 8 ,'minimum_decreasing_count': 6, 'max_different_atr_from_MACD': 0.2, 'max_ATR_to_for_consolidation': 0.3 , 'min_time_for_consolidation': 4},
    '4h': {'name': '4h', 'dataOffset': pd.Timedelta(hours=4), 'minimum_increasing_count': 8 ,'minimum_decreasing_count': 6, 'max_different_atr_from_MACD': 0.2, 'max_ATR_to_for_consolidation': 0.3 , 'min_time_for_consolidation': 4},
    '2h': {'name': '2h', 'dataOffset': pd.Timedelta(hours=2), 'minimum_increasing_count': 8 ,'minimum_decreasing_count': 6, 'max_different_atr_from_MACD': 0.2, 'max_ATR_to_for_consolidation': 0.35 , 'min_time_for_consolidation': 5},
    '1h': {'name': '1h', 'dataOffset': pd.Timedelta(hours=1), 'minimum_increasing_count': 8 ,'minimum_decreasing_count': 6, 'max_different_atr_from_MACD': 0.2, 'max_ATR_to_for_consolidation': 0.35 , 'min_time_for_consolidation': 5},
    '30m': {'name': '30m', 'dataOffset': pd.Timedelta(minutes=30), 'minimum_increasing_count': 8 ,'minimum_decreasing_count': 6, 'max_different_atr_from_MACD': 0.25, 'max_ATR_to_for_consolidation': 0.5 , 'min_time_for_consolidation': 5},
    '15m': {'name': '15m', 'dataOffset': pd.Timedelta(minutes=15), 'minimum_increasing_count': 8 ,'minimum_decreasing_count': 6, 'max_different_atr_from_MACD': 0.25, 'max_ATR_to_for_consolidation': 0.5 , 'min_time_for_consolidation': 5} ,
    '10m': {'name': '10m', 'dataOffset': pd.Timedelta(minutes=10), 'minimum_increasing_count': 8 ,'minimum_decreasing_count': 6, 'max_different_atr_from_MACD': 0.25, 'max_ATR_to_for_consolidation': 0.55 , 'min_time_for_consolidation': 5} ,
    '5m': {'name': '5m', 'dataOffset': pd.Timedelta(minutes=5), 'minimum_increasing_count': 8 ,'minimum_decreasing_count': 6, 'max_different_atr_from_MACD': 0.25, 'max_ATR_to_for_consolidation': 0.55 , 'min_time_for_consolidation': 5} ,
    '3m': {'name': '3m', 'dataOffset': pd.Timedelta(minutes=3), 'minimum_increasing_count': 10 ,'minimum_decreasing_count': 8, 'max_different_atr_from_MACD': 0.25, 'max_ATR_to_for_consolidation': 0.55 , 'min_time_for_consolidation': 5} ,
    '1m': {'name': '1m', 'dataOffset': pd.Timedelta(minutes=1), 'minimum_increasing_count': 10 ,'minimum_decreasing_count': 10, 'max_different_atr_from_MACD': 0.25, 'max_ATR_to_for_consolidation': 0.12 , 'min_time_for_consolidation': 5}  
}




def add_williams_R(df):

    df['14_day_high'] = df['high'].rolling(window=14).max()
    df['14_day_low'] = df['low'].rolling(window=14).min()

    # Calculate the Williams %R
    df['Williams_%R'] = (df['14_day_high'] - df['close']) / (df['14_day_high'] - df['14_day_low']) * -100
    return df


def calculate_stochrsi(df, period=14):
    # Minimalna i maksymalna wartość RSI z ostatnich 'period' okresów
    min_rsi = df['RSI'].rolling(window=period).min()
    max_rsi = df['RSI'].rolling(window=period).max()

    # Obliczanie Stochastic RSI
    df['Stoch_RSI'] = (df['RSI'] - min_rsi) / (max_rsi - min_rsi)
    
    return df

def calculate_stoch_rsi_k_d(df, rsi_period=14, k_period=3):
    # Oblicz Stochastic RSI (%K)

    df = calculate_stochrsi(df, period=rsi_period)
    
    # Oblicz %D jako SMA z %K
    df['K'] = df['Stoch_RSI'].rolling(window=k_period).mean()*100
    df['D'] = df['K'].rolling(window=k_period).mean()
    return df


def add_price_action(df):
    df['_Srednia_Kroczaca_Wolumenu'] = df['volume'].rolling(window=40).mean()
    df['_Diff_High_low'] = df['high'] - df['low']

    df['Duzy_wolumen'] = ((df['volume'] > 1.8 * df['_Srednia_Kroczaca_Wolumenu'])).astype(int)
    
    df['Dluga_swieczka'] = ((df['_Diff_High_low'] > 1.2 * df['ATR'])).astype(int)
    df['Kierunek_wzrostowy'] = ((df['close'] - df['low'] > df['_Diff_High_low'] *0.7 )).astype(int)
    df['Kierunek_spadkowy'] = ((df['high'] - df['close'] > df['_Diff_High_low'] *0.7 )).astype(int)

    df['Formacja_wzrostowa'] = ((df['Duzy_wolumen'] & df['Dluga_swieczka'] & df['Kierunek_wzrostowy'] )).astype(int)
    df['Formacja_spadkowa'] = ((df['Duzy_wolumen'] & df['Dluga_swieczka'] & df['Kierunek_spadkowy'] )).astype(int)

    return df
def add_emas(data):
    data['EMA_10'] = data['close'].ewm(span=10, adjust=False).mean()
    data['EMA_20'] = data['close'].ewm(span=20, adjust=False).mean()
    data['EMA_50'] = data['close'].ewm(span=50, adjust=False).mean()
    data['EMA_100'] = data['close'].ewm(span=100, adjust=False).mean()
    data['EMA_200'] = data['close'].ewm(span=200, adjust=False).mean()
    data['EMA_500'] = data['close'].ewm(span=500, adjust=False).mean()
    data['EMA_595'] = data['close'].ewm(span=595, adjust=False).mean()
    return data
def add_smas(data):
    data['SMA_20'] = data['close'].rolling(window=20).mean()
    data['SMA_100'] = data['close'].rolling(window=100).mean()
    data['SMA_150'] = data['close'].rolling(window=150).mean()
    data['SMA_200'] = data['close'].rolling(window=200).mean()
    data['SMA_500'] = data['close'].rolling(window=500).mean()
    data['SMA_592'] = data['close'].rolling(window=592).mean()
    return data

def blueWaves_vwap(src, chlLen, avgLen):
    esa = talib.EMA(src, chlLen)
    d = talib.EMA(np.abs(src - esa), chlLen)
    ci = (src - esa) / (0.015 * d)
    bw1 = talib.EMA(ci, avgLen)
    bw2 = talib.SMA(bw1, 3)
    vwap = bw1 - bw2


    return bw1, bw2, vwap

def cond( data):
    hlc3 = (data['high'] + data['low'] + data['close']) / 3
    bw1, bw2, vwap = blueWaves_vwap(hlc3, 9, 12)
    return bw1, bw2, vwap



def calculate_trend(row):
    if row['m1'] > row['m2'] and row['m1_prev'] < row['m2_prev']:
        return 1
    elif row['m1'] < row['m2'] and row['m1_prev'] > row['m2_prev']:
        return -1
    else:
        return 0
    


def add_indicators_macd(df):

    short_window = 12
    long_window = 26
    signal_window = 9
    if 'MACD_Histogram' in df.columns and (df['MACD_Histogram'].last_valid_index() +1 ==len(df)):
        return df
    else:
        df['EMA12'] = df['close'].ewm(span=short_window).mean()
        df['EMA26'] = df['close'].ewm(span=long_window).mean()

        # Obliczanie MACD Line
        df['MACD'] = df['EMA12'] - df['EMA26']

        # Obliczanie Signal Line
        df['Signal_Line'] = df['MACD'].ewm(span=signal_window).mean()

        # Opcjonalnie, obliczanie Histogramu MACD
        df['MACD_Histogram'] = df['MACD'] - df['Signal_Line']     
        df['Signal'] = 0

    # Warunki do określenia przecięć
        df.loc[(df['MACD'] > df['Signal_Line']) & (df['MACD'].shift(1) <= df['Signal_Line'].shift(1)), 'Signal_MACD'] = 1
        df.loc[(df['MACD'] < df['Signal_Line']) & (df['MACD'].shift(1) >= df['Signal_Line'].shift(1)), 'Signal_MACD'] = -1 

        df['trend_MACD'] = df['MACD_Histogram'].apply(lambda x: 1 if x > 0 else -1)


        return df

def add_indicators_smma(data, length=7):
    if 'SMMA_7' in data.columns and (data['SMMA_7'].last_valid_index() +1 ==len(data)):
        return data
    else:
        data['SMMA_7']=data['close'].ewm(alpha=1/length, adjust=False).mean()
        return data

def add_indicators_cipher(data,since=pd.to_datetime('2000-01-01 00:00:00')):

    required_columns = ['m1', 'm2','vW']
    if all(col in data.columns for col in required_columns):
        data=data.drop(['m1', 'm2', 'vW', 'm1_prev', 'm2_prev'], axis=1)

    b1, b2, vW = cond( data)
    b1.name = 'm1'  # Nadaj nazwę kolumnie bw1H1
    data_with_indicators = data.join(b1)
    b2.name = 'm2'  # Nadaj nazwę kolumnie bw1H1
    data_with_indicators = data_with_indicators.join(b2)
    vW.name = 'vW'  # Nadaj nazwę kolumnie bw1H1
    data_with_indicators = data_with_indicators.join(vW)
    # Utwórz kolumny z opóźnionymi wartościami m1 i m2
    m1_prev = data_with_indicators['m1'].shift(1)
    m1_prev.name = 'm1_prev'  # Nadaj nazwę kolumnie bw1H1
    m2_prev = data_with_indicators['m2'].shift(1)
    m2_prev.name = 'm2_prev'  # Nadaj nazwę kolumnie bw1H1
    data_with_indicators = data_with_indicators.join(m1_prev)
    data_with_indicators = data_with_indicators.join(m2_prev)
    # Zainicjuj zmienną trendu
    trend_value = 0
    data_with_indicators["MC_signal"]=0
    # Przypisz wartości do kolumny trendu
    for i, row in data_with_indicators.iterrows():
        if row['m1'] > row['m2'] and row['m1_prev'] < row['m2_prev']:
            trend_value = 1
            data_with_indicators.at[i, 'MC_signal'] = 1
        elif row['m1'] < row['m2'] and row['m1_prev'] > row['m2_prev']:
            trend_value = -1
            data_with_indicators.at[i, 'MC_signal'] = -1
        data_with_indicators.at[i, 'trend_MC'] = trend_value
        data_with_indicators.at[i, 'trend_MC'] = trend_value

    return data_with_indicators


def calculate_true_range(df):
    """Calculate the True Range (TR)"""
    df['previous_close'] = df['close'].shift(1)
    df['TR'] = np.maximum(df['high'] - df['low'], 
                          np.maximum(abs(df['high'] - df['previous_close']), 
                                     abs(df['low'] - df['previous_close'])))
    return df

def sma(series, length):
    """Simple Moving Average (SMA)"""
    return series.rolling(window=length).mean()

def ema(series, length):
    """Exponential Moving Average (EMA)"""
    return series.ewm(span=length, adjust=False).mean()

def wma(series, length):
    """Weighted Moving Average (WMA)"""
    weights = np.arange(1, length + 1)
    return series.rolling(length).apply(lambda prices: np.dot(prices, weights) / weights.sum(), raw=True)

def rma(series, length):
    """Relative Moving Average (RMA)"""
    return series.ewm(alpha=1/length, adjust=False).mean()


def ma(source, length, ma_type):
    """Funkcja do wyznaczania odpowiedniej średniej kroczącej do RSI """ 
    if ma_type == "SMA":
        return sma(source, length)
    elif ma_type == "EMA":
        return ema(source, length)
    elif ma_type == "WMA":
        return wma(source, length)
    elif ma_type == "SMMA (RMA)":
        return rma(source, length)
    else:
        raise ValueError("Unsupported MA type")
    
def ma_function(series, length, smoothing):
    
    if smoothing == "SMA":
        return sma(series, length)
    elif smoothing == "EMA":
        return ema(series, length)
    elif smoothing == "WMA":
        return wma(series, length)
    else:  # Default to RMA
        return rma(series, length)
    


def calculate_rsi(df, rsi_length=14, ma_type="SMA", ma_length=14, bb_mult=2):
    delta = df['close'].diff()
    gain = delta.where(delta > 0, 0)
    loss = -delta.where(delta < 0, 0)

    # Relative Strength (RS)
    avg_gain = rma(gain, rsi_length)
    avg_loss = rma(loss, rsi_length)
    rs = avg_gain / avg_loss

    # RSI calculation
    rsi = 100 - (100 / (1 + rs))

    # Obliczenie średniej ruchomej RSI (wg wybranego typu MA)
    rsi_ma = ma(rsi, ma_length, ma_type)

    # Bollinger Bands
    is_bb = (ma_type == "Bollinger Bands")
    upper_bb = rsi_ma + bb_mult * rsi.rolling(window=ma_length).std() if is_bb else np.nan
    lower_bb = rsi_ma - bb_mult * rsi.rolling(window=ma_length).std() if is_bb else np.nan

    df['RSI'] = rsi
    df['RSI_MA'] = rsi_ma
    df['Upper_BB'] = upper_bb
    df['Lower_BB'] = lower_bb

    return df

# ATR calculation
def calculate_atr(df, length=14, smoothing="RMA"):
    df = calculate_true_range(df)
    df['ATR'] = ma_function(df['TR'], length, smoothing)
    return df



def calculate_obv(data, span1=1, span2=3):
    """
    Funkcja do obliczania wskaźnika OBV (On-Balance Volume).
    
    :param data: DataFrame zawierający kolumny 'Close' i 'Volume'
    :return: DataFrame z dodatkową kolumną 'OBV'
    """
    # Tworzymy kolumnę 'OBV' i inicjalizujemy pierwszą wartość jako 0
    data['OBV'] = 0
    for i in range(1, len(data)):
        if data['close'][i] > data['close'][i - 1]:
            data.loc[i, 'OBV'] = data.loc[i - 1, 'OBV'] + data.loc[i, 'volume']
        elif data['close'][i] < data['close'][i - 1]:
            data.loc[i, 'OBV'] = data.loc[i - 1, 'OBV'] - data.loc[i, 'volume']
        else:
            data.loc[i, 'OBV'] = data.loc[i - 1, 'OBV']

    data['OBV_ExK1'] = data['OBV'].ewm(span=span1, adjust=False).mean()
    data['OBV_ExK2'] = data['OBV'].ewm(span=span2, adjust=False).mean()
    data['OBV_ExDiff'] = data['OBV_ExK1'] - data['OBV_ExK2']
    return data
def calculate_long_term_obv(data, period=10):
    """
    Wyznacza długoterminowy trend za pomocą średniej ruchomej.
    :param data: DataFrame zawierający kolumnę 'OBV'
    :param period: Liczba świeczek do uwzględnienia w trendzie
    :return: DataFrame z nową kolumną 'LongTermTrend'
    """
    data['OBV_LT'] = data['OBV_ExDiff'].ewm(span=period, adjust=False).mean()
    return data



In [7]:
def get_local_data(ticker, interval, folder):
    # Replace '/' with '.' in the ticker symbol
    symbol = ticker.replace('/', '.')
    
    # Create the directory path
    symbol_directory = os.path.join(folder, symbol)
    
    # Create the directory if it doesn't exist
    if not os.path.exists(symbol_directory):
        os.makedirs(symbol_directory)
    
    # Create the file path
    file_path = os.path.join(symbol_directory, interval + '.csv')
    
    # Load the data from the CSV file if it exists
    df = pd.read_csv(file_path)

    if 'timestamp' not in df.columns:
        if 'index' in df.columns:
            print(f"🔁 Zamieniono kolumnę 'index' na 'timestamp' w pliku: {file_path}")
            df.rename(columns={'index': 'timestamp'}, inplace=True)
            df.to_csv(file_path, index=False)  # Nadpisz poprawiony plik
        else:
            print(f"❌ Ani 'timestamp', ani 'index' nie znaleziono w pliku: {file_path}")
            return None

    # Wczytaj poprawiony plik z parsowaniem daty
    df = pd.read_csv(file_path, parse_dates=['timestamp'])
    return df

def process_data(df):
    processed_data=df
    #processed_data=add_indicators_cipher(processed_data)
    processed_data=add_indicators_macd(processed_data)

    processed_data=calculate_obv(processed_data)
    processed_data=calculate_long_term_obv(processed_data)

    processed_data = calculate_atr(processed_data, 14, 'RMA')

    processed_data=add_indicators_macd(processed_data)
    processed_data=add_price_action(processed_data)
    processed_data=calculate_rsi(processed_data)
    processed_data= calculate_stoch_rsi_k_d(processed_data)

    processed_data=add_williams_R(processed_data)
    processed_data['Williams_%R_lag'] = processed_data['Williams_%R'].shift(1)
    processed_data=add_emas(processed_data)
    processed_data=add_smas(processed_data)
    return processed_data

In [8]:
class Trade:
    def __init__(self, trade_type, create_time, entry_time, entry_price, deposit, max_levar, strategy, row, stop_loss=None): # stop_loss, take_profit, ):
        self.type = trade_type
        self.create_time=create_time
        self.entry_time = entry_time
        self.entry_price = entry_price
        self.deposit= deposit
        if stop_loss:
            self.position_size= deposit/ abs(self.entry_price-self.stop_loss)
            self.position_value= self.position_size*entry_price 
            self.levar = self.position_value/deposit  
            if(self.levar>max_levar):
                self.levar=max_levar
                self.position_value=deposit*max_levar
                self.position_size= self.position_value/self.entry_price
        else:
            self.levar=1
            self.position_size= deposit/entry_price
            self.position_value= self.position_size*entry_price
        self.strategy= strategy
        self.row=row
        self.stop_loss=stop_loss


        self.exit_time = None
        self.exit_price = None
        self.pnl = None
        self.result = None

        self.exit_value=None
        self.status = "Active"  # Domyślnie ustawiamy status na "Active"
    
    def set_exit(self, exit_time, exit_price):
        self.exit_time = exit_time
        self.exit_price = exit_price
        self.pnl = self.calculate_pnl()
        self.exit_value= self.calculate_exit_value()
        self.status = "Closed"  # Ustawiamy status na "Closed" po ustawieniu exit_time i exit_price

    def calculate_pnl(self):
        if(self.type=='L'):
                return (self.exit_price - self.entry_price) * self.position_size

        if(self.type=='S'):
                return(self.entry_price - self.exit_price ) * self.position_size

    def calculate_exit_value(self):
        prowizja= self.entry_price*self.position_size*PROWIZJA
        return self.deposit + self.pnl - prowizja*2

    def calculate_current_value(self, price):
        if(self.type=='L'):
            return (price - self.entry_price) * self.position_size + self.deposit
        if(self.type=='S'):
            return (self.entry_price - price ) * self.position_size + self.deposit 


    def set_status(self, status):
        if status in ["Active", "Partially_Closed", "Closed"]:
            self.status = status
        else:
            raise ValueError("Invalid status. Status must be one of: 'Active', 'Partially_Closed', 'Closed'.")
    
    def to_dict(self):
        base_data = {
            'Type': self.type,
            'Create_time': self.create_time,
            'Entry Time': self.entry_time,
            'Entry Price': self.entry_price,
            'Exit Time': self.exit_time,
            'Exit Price': self.exit_price,
            'result': self.result,
            'P&L': self.pnl,
            'Levar': self.levar,
            'Deposit': self.deposit
        }

        # Dodanie snapshotu z danych rynkowych
        market_data_at_entry = {
            f"{key} at entry": value for key, value in self.row.items()
        }

        return {**base_data, **market_data_at_entry}
    
    
    
    def __str__(self):
        return f"{self.type} trade opened at {self.entry_time} with entry price {self.entry_price} and {self.position_size} units.  Status: {self.status}"


In [9]:
class Order:
    def __init__(self, order_type, trade_type, create_time,  deposit, max_levar,strategy,  row):
        self.order_type=order_type
        self.trade_type = trade_type  # Typ transakcji (np. "Long", "Short")
        self.create_time=create_time
        # self.entry_price = entry_price  # Cena wejścia
        # self.stop_loss = stop_loss  # 
        # self.take_profit= take_profit
        # self.levar = levar  # Dźwignia finansowa
        # self.balance = balance  # Saldo konta
        # self.available_balance=available_balance
        self.deposit= deposit
        # self.interval=interval
        # self.expired_time=self.create_time+interval_mapping[interval]['dataOffset'] * 5
        self.status='CREATED'
        self.strategy=strategy
        self.max_levar= max_levar
        self.row=row
        

    def fill(self, entry_time, entry_price):

        new_trade = Trade(
                    trade_type=self.trade_type,
                    create_time=self.create_time,
                    entry_time=entry_time,
                    entry_price=entry_price,
                    deposit=self.deposit,
                    max_levar=self.max_levar,
                    strategy=self.strategy,
                    row=self.row  # snapshot rynku
                    # take_profit=self.take_profit,
                    # stop_loss=self.stop_loss,
                    # interval=self.interval,
                )

        self.status = 'FILLED'
        return new_trade
    def __str__(self):
        return f"Order: type: {self.order_type} type: {self.trade_type} create_time: {self.create_time} deposit: {self.deposit} strategy: {self.strategy} "

In [10]:
def update_account_history(account_history: pd.DataFrame, timestamp, account_value):
    new_row = pd.DataFrame({'timestamp': [timestamp], 'account_value': [account_value]})
    return pd.concat([account_history, new_row], ignore_index=True)

def create_market_open_order(trade_type, deposit, row, strategy, max_levar=1):
    logging.info(f" row timestamp: {row['timestamp']} ")
    order_params = {
                            'order_type': 'OPEN MARKET',
                            'trade_type': trade_type,
                            'create_time': row['timestamp'],
                            'deposit': deposit,
                            'max_levar': max_levar,
                            'strategy': strategy,
                            'row': row
                        }
    order = Order(**order_params)
    return order

In [11]:

PROWIZJA=0.01*0.1  # 0.01 % prowizji 


Risk_size=0.002


def calculate_deposit(initial_balance, available_balance, stop_loss = None):
    if stop_loss:
        return initial_balance*Risk_size
    else:
        return available_balance
                      

DAILY_UPDATE=True
LOG=True

def backtest(interval_dict, ticker, strategy, intervals,  start_date, end_date , start_balance=10000, position_risk=0.002, stop_loss=False):

    initial_balance=start_balance
    balance = start_balance  # Stan początkowy konta handlowego
    available_balance=balance
    account_worth= balance
    curr_date=start_date

    account_history = pd.DataFrame(columns=['timestamp', 'account_value'])
    Active_orders = []
    Closed_trades = []
    Active_trades = []
    active_long=0
    active_short=0

    logging.info(f"Starting singleinterval simulation {intervals} for ticker {ticker} strategy {strategy} for date: {start_date} - {end_date} ")
    #print(f"Starting singleinterval simulation {intervals} for ticker {ticker} strategy {strategy} for date: {start_date} - {end_date} ")
    main_interval=interval_dict[intervals[0]]
    curr_date=start_date
    main_interval.set_date(curr_date) # Setting the current date ( or nearest existing date) for interval
    if(main_interval.curr_date is None):
        logging.info(f"Empty data for interval {main_interval.interval} for date {curr_date}")
        return None, None
    
    curr_date=main_interval.curr_date # Uploading current date to existing in data
    while  curr_date <= end_date:
        #Set the current date for interval
        if DAILY_UPDATE:
            logging.info(f"Starting iteration for date: {curr_date} ")
        #logging.info(f"Starting iteration for date: {curr_date} ")
        #main_interval.set_date(curr_date)
        row=main_interval.row
        if (row['timestamp']!=curr_date): #ELack of data for current date. Skipping
            logging.info(f" ERROR:  row timestamp: {row['timestamp']}  curr date: {curr_date} ")
            #print(f" ERROR:  row timestamp: {row['timestamp']}  curr date: {curr_date} ")
            curr_date += interval_mapping[main_interval.interval]['dataOffset']
            continue
            #return None, None
        if(main_interval.curr_date is None): #End of data
            logging.info(f"ERROR Empty data for interval {main_interval.interval} for date {curr_date}. End of data!")
            #print(f" ERROR:  row timestamp: {row['timestamp']}  curr date: {curr_date} ")

            return None, None
        #Processing orders
        if Active_orders:
            do_usuniecia = []
            for order in Active_orders:
                logging.info(f"Processing order: {order.order_type} {order.status} {order.create_time} ")
                if order.order_type=='OPEN MARKET' and order.status =='CREATED' and order.create_time<curr_date:
                    entry_time = curr_date
                    entry_price = main_interval.data.at[main_interval.curr_idx, 'open']
                    new_trade = order.fill(entry_time, entry_price)
                    do_usuniecia.append(order)
                    
                    if LOG:
                        logging.info(f"New trade created: {new_trade}")

                    Active_trades.append(new_trade)
                    balance -= new_trade.deposit

            for order in do_usuniecia:
                Active_orders.remove(order)
                if LOG:
                    logging.info(f"Order removed: {order} , BALANCE: {balance} AVAILABLE_BALANCE: {available_balance}")

        #Processing current trades. Looking for new trades ( setting orders)
        # LOOKING FOR TRADES
        if strategy=='0.0':  # Buy&Hold strategy
            if active_long==0: # Enetring long
                    deposit=calculate_deposit(initial_balance, available_balance)
                    if available_balance>=deposit:
                        new_order=create_market_open_order("L" , deposit, row.copy(), strategy)
                        Active_orders.append(new_order)
                        active_long=1
                        available_balance -= new_order.deposit 
        if strategy=='0.1':  # First strtategy: SMA, MACD, StochRSI
            if row['SMA_20'] > row['SMA_150']: # 
                if active_short==1: # Exit short
                    trade=Active_trades[0] # Only one trade
                    active_short=0
                    exit_price = row['close']  # Cena wyjścia
                    trade.set_exit( exit_time= curr_date, exit_price= exit_price)
                    available_balance += trade.exit_value
                    trade.result = "SIGNAL"
                    Active_trades.remove(trade)
                    Closed_trades.append(trade)
                    if LOG:
                        logging.info(f"Exited trade: {trade}, available balance: {available_balance}")
                        logging.info(f"row['D'] : {row['D'] }, row['MACD_Histogram'] {row['MACD_Histogram']}")

                if active_long==0 and row['SMA_20'] > row['SMA_150'] and row['D']  <= 20 and row['MACD_Histogram'] >0: # Enetring long
                    deposit=calculate_deposit(initial_balance, available_balance)
                    if available_balance>=deposit:
                        new_order=create_market_open_order("L" , deposit, row.copy(), strategy)
                        Active_orders.append(new_order)
                        active_long=1
                        available_balance -= new_order.deposit 
                        if LOG:
                            logging.info(f"New order created: {new_order} , available balance: {available_balance}")
                            
            if row['SMA_20'] < row['SMA_150']: # 
                if active_long==1: # Exit long
                    trade=Active_trades[0] # Only one trade
                    active_long=0
                    exit_price = row['close']  # Cena wyjścia
                    trade.set_exit( exit_time= curr_date, exit_price= exit_price)
                    trade.result = "SIGNAL"
                    available_balance += trade.exit_value
                    Active_trades.remove(trade)
                    Closed_trades.append(trade)
                    if LOG:
                        logging.info(f"Exited trade: {trade}, available balance: {available_balance}")

                if active_short==0 and row['SMA_20'] < row['SMA_150'] and row['D']  >=80  and row['MACD_Histogram'] <0: # Enetring short
                    deposit=calculate_deposit(initial_balance, available_balance)
                    if available_balance>=deposit:

                        new_order=create_market_open_order("S" , deposit, row.copy(), strategy)
                        Active_orders.append(new_order)
                        active_short=1
                        available_balance -= new_order.deposit 
                        if LOG:
                            logging.info(f"New order created: {new_order} , available balance: {available_balance}")

        if strategy=='0.2': # Second strategy: MACD
            if row['Signal_MACD']==1: # 
                if active_short==1: # Exit short
                    trade=Active_trades[0] # Only one trade
                    active_short=0
                    exit_price = row['close']  # Cena wyjścia
                    trade.set_exit( exit_time= curr_date, exit_price= exit_price)
                    available_balance += trade.exit_value
                    trade.result = "SIGNAL"
                    Active_trades.remove(trade)
                    Closed_trades.append(trade)
                    if LOG:
                        logging.info(f"Exited trade: {trade}, available balance: {available_balance}")
                        logging.info(f"row['D'] : {row['D'] }, row['MACD_Histogram'] {row['MACD_Histogram']}")

                if active_long==0 : # Enetring long
                    deposit=calculate_deposit(initial_balance, available_balance)
                    if available_balance>=deposit:
                        new_order=create_market_open_order("L" , deposit, row.copy(), strategy)
                        Active_orders.append(new_order)
                        active_long=1
                        available_balance -= new_order.deposit 
                        if LOG:
                            logging.info(f"New order created: {new_order} , available balance: {available_balance}")
                            
            if row['Signal_MACD']==-1: # 
                if active_long==1: # Exit long
                    trade=Active_trades[0] # Only one trade
                    active_long=0
                    exit_price = row['close']  # Cena wyjścia
                    trade.set_exit( exit_time= curr_date, exit_price= exit_price)
                    trade.result = "SIGNAL"
                    available_balance += trade.exit_value
                    Active_trades.remove(trade)
                    Closed_trades.append(trade)
                    if LOG:
                        logging.info(f"Exited trade: {trade}, available balance: {available_balance}")

                if active_short==0 : # Enetring short
                    deposit=calculate_deposit(initial_balance, available_balance)
                    if available_balance>=deposit:

                        new_order=create_market_open_order("S" , deposit, row.copy(), strategy)
                        Active_orders.append(new_order)
                        active_short=1
                        available_balance -= new_order.deposit 
                        if LOG:
                            logging.info(f"New order created: {new_order} , available balance: {available_balance}")



        if strategy=='0.3': # Third strategy: EMA(20), close. LONG oonly
            if row['close'] > row['EMA_20']: # 

                if active_long==0 : # Enetring long
                    deposit=calculate_deposit(initial_balance, available_balance)
                    if available_balance>=deposit:
                        new_order=create_market_open_order("L" , deposit, row.copy(), strategy)
                        Active_orders.append(new_order)
                        active_long=1
                        available_balance -= new_order.deposit 
                        if LOG:
                            logging.info(f"New order created: {new_order} , available balance: {available_balance}")
                            
            if row['close'] < row['EMA_20']: # 
                if active_long==1: # Exit long
                    trade=Active_trades[0] # Only one trade
                    active_long=0
                    exit_price = row['close']  # Cena wyjścia
                    trade.set_exit( exit_time= curr_date, exit_price= exit_price)
                    trade.result = "SIGNAL"
                    available_balance += trade.exit_value
                    Active_trades.remove(trade)
                    Closed_trades.append(trade)
                    if LOG:
                        logging.info(f"Exited trade: {trade}, available balance: {available_balance}")

        if strategy=='0.4': # Fourth strategy: EMA(10), SMA_100, RSI for entering Short
            if row['EMA_10']>=row['SMA_100']: # 
                if active_short==1: # Exit short
                    trade=Active_trades[0] # Only one trade
                    active_short=0
                    exit_price = row['close']  # Cena wyjścia
                    trade.set_exit( exit_time= curr_date, exit_price= exit_price)
                    available_balance += trade.exit_value
                    trade.result = "SIGNAL"
                    Active_trades.remove(trade)
                    Closed_trades.append(trade)
                    if LOG:
                        logging.info(f"Exited trade: {trade}, available balance: {available_balance}")
                        logging.info(f"row['D'] : {row['D'] }, row['MACD_Histogram'] {row['MACD_Histogram']}")

                if active_long==0 : # Enetring long
                    deposit=calculate_deposit(initial_balance, available_balance)
                    if available_balance>=deposit:
                        new_order=create_market_open_order("L" , deposit, row.copy(), strategy)
                        Active_orders.append(new_order)
                        active_long=1
                        available_balance -= new_order.deposit 
                        if LOG:
                            logging.info(f"New order created: {new_order} , available balance: {available_balance}")
                            
            if row['EMA_10']<row['SMA_100']: # 
                if active_long==1: # Exit long
                    trade=Active_trades[0] # Only one trade
                    active_long=0
                    exit_price = row['close']  # Cena wyjścia
                    trade.set_exit( exit_time= curr_date, exit_price= exit_price)
                    trade.result = "SIGNAL"
                    available_balance += trade.exit_value
                    Active_trades.remove(trade)
                    Closed_trades.append(trade)
                    if LOG:
                        logging.info(f"Exited trade: {trade}, available balance: {available_balance}")

                if active_short==0 and row['RSI']<50 : # Enetring short, extra condition RSI<50
                    deposit=calculate_deposit(initial_balance, available_balance)
                    if available_balance>=deposit:

                        new_order=create_market_open_order("S" , deposit, row.copy(), strategy)
                        Active_orders.append(new_order)
                        active_short=1
                        available_balance -= new_order.deposit 
                        if LOG:
                            logging.info(f"New order created: {new_order} , available balance: {available_balance}")

        if strategy=='0.5': # Fifth strategy: Williams%R
            if row['Williams_%R'] <-80: # 
                if active_short==1: # Exit short
                    trade=Active_trades[0] # Only one trade
                    active_short=0
                    exit_price = row['close']  # Cena wyjścia
                    trade.set_exit( exit_time= curr_date, exit_price= exit_price)
                    available_balance += trade.exit_value
                    trade.result = "SIGNAL"
                    Active_trades.remove(trade)
                    Closed_trades.append(trade)
                    if LOG:
                        logging.info(f"Exited trade: {trade}, available balance: {available_balance}")
                        logging.info(f"row['D'] : {row['D'] }, row['MACD_Histogram'] {row['MACD_Histogram']}")

                if active_long==0 : # Enetring long
                    deposit=calculate_deposit(initial_balance, available_balance)
                    if available_balance>=deposit:
                        new_order=create_market_open_order("L" , deposit, row.copy(), strategy)
                        Active_orders.append(new_order)
                        active_long=1
                        available_balance -= new_order.deposit 
                        if LOG:
                            logging.info(f"New order created: {new_order} , available balance: {available_balance}")
                            
            if row['Williams_%R'] >-20: # 
                if active_long==1: # Exit long
                    trade=Active_trades[0] # Only one trade
                    active_long=0
                    exit_price = row['close']  # Cena wyjścia
                    trade.set_exit( exit_time= curr_date, exit_price= exit_price)
                    trade.result = "SIGNAL"
                    available_balance += trade.exit_value
                    Active_trades.remove(trade)
                    Closed_trades.append(trade)
                    if LOG:
                        logging.info(f"Exited trade: {trade}, available balance: {available_balance}")

                if active_short==0 : # Enetring short
                    deposit=calculate_deposit(initial_balance, available_balance)
                    if available_balance>=deposit:

                        new_order=create_market_open_order("S" , deposit, row.copy(), strategy)
                        Active_orders.append(new_order)
                        active_short=1
                        available_balance -= new_order.deposit 
                        if LOG:
                            logging.info(f"New order created: {new_order} , available balance: {available_balance}")


        if strategy=='0.6': # Sixth strategy: Williams%R, Candle pattern
            if   (row['Williams_%R_lag'] < -80 or row['Williams_%R'] < -80) : #
                if active_short==1: # Exit short
                    trade=Active_trades[0] # Only one trade
                    active_short=0
                    exit_price = row['close']  # Cena wyjścia
                    trade.set_exit( exit_time= curr_date, exit_price= exit_price)
                    available_balance += trade.exit_value
                    trade.result = "SIGNAL"
                    Active_trades.remove(trade)
                    Closed_trades.append(trade)
                    if LOG:
                        logging.info(f"Exited trade: {trade}, available balance: {available_balance}")
                        logging.info(f"row['D'] : {row['D'] }, row['MACD_Histogram'] {row['MACD_Histogram']}")

                if active_long==0 and row['Formacja_wzrostowa'] == 1  : # Enetring long
                    deposit=calculate_deposit(initial_balance, available_balance)
                    if available_balance>=deposit:
                        new_order=create_market_open_order("L" , deposit, row.copy(), strategy)
                        Active_orders.append(new_order)
                        active_long=1
                        available_balance -= new_order.deposit 
                        if LOG:
                            logging.info(f"New order created: {new_order} , available balance: {available_balance}")
                            
            if  (row['Williams_%R_lag'] >-20 or row['Williams_%R'] >-20) : # 
                if active_long==1 and Active_trades: # Exit long
                    trade=Active_trades[0] # Only one trade
                    active_long=0
                    exit_price = row['close']  # Cena wyjścia
                    trade.set_exit( exit_time= curr_date, exit_price= exit_price)
                    trade.result = "SIGNAL"
                    available_balance += trade.exit_value
                    Active_trades.remove(trade)
                    Closed_trades.append(trade)
                    if LOG:
                        logging.info(f"Exited trade: {trade}, available balance: {available_balance}")

                if active_short==0 and row['Formacja_spadkowa'] == 1  : # Enetring short
                    deposit=calculate_deposit(initial_balance, available_balance)
                    if available_balance>=deposit:

                        new_order=create_market_open_order("S" , deposit, row.copy(), strategy)
                        Active_orders.append(new_order)
                        active_short=1
                        available_balance -= new_order.deposit 
                        if LOG:
                            logging.info(f"New order created: {new_order} , available balance: {available_balance}")


                    # Calculate account worth. Save to account history
        account_worth=available_balance

        for trade in Active_trades:
            account_worth+=trade.calculate_current_value(row['close'])
        for order in Active_orders:
            account_worth+=order.deposit
        account_history = update_account_history(account_history, curr_date, account_worth)
                    # Set the next date
        curr_date += interval_mapping[main_interval.interval]['dataOffset']
        main_interval.set_next_date()


     # END OF BACKTESTING
     # CLOSING ACTIVE TRADES
             
    if Active_trades:
        for trade in Active_trades[:]:  
            trade.set_exit(exit_time=curr_date, exit_price=row['close'])
            trade.result = "END"
            available_balance += trade.exit_value

            if LOG:
                logging.info(f"Exited trade: {trade}, available balance: {available_balance}")

            Active_trades.remove(trade)
            Closed_trades.append(trade)

    closed_trades_df= pd.DataFrame()
# Sprawdzenie, czy Closed_trades jest pusty
    if Closed_trades:
        closed_trades_df = pd.DataFrame([trade.to_dict() for trade in Closed_trades])
    
        closed_trades_df = closed_trades_df.sort_values(by='Entry Time', ascending=True)


    #print("Balans na koniec:", available_balance)

    return closed_trades_df, account_history


In [12]:
class Interval:
    def __init__(self, ticker, interval, data):
        self.ticker=ticker
        self.interval=interval
        self.data=data
        self.curr_idx=None 
        self.curr_date=None
        self.row=None
    def set_date(self, date):
        self.curr_idx = (self.data['timestamp'] - date).abs().idxmin()
        if self.curr_idx!=None:
            self.row=self.data.iloc[self.curr_idx]
            self.curr_date=self.row['timestamp']
           # logging.info(f"Date {date} found in data idx: {self.curr_idx}")
        #else:
            #logging.info(f"Date {date} not found in data")
    

    def set_next_date(self): # Next row if exists
        self.curr_idx+=1

        if 0 <= self.curr_idx < len(self.data):
            self.row = self.data.iloc[self.curr_idx]
            self.curr_date=self.row['timestamp']
        else:
            self.row = None  # If the index is out of range, set the row to None
            self.curr_date=None
    





In [ ]:




# Wczytaj wszystkie arkusze z pliku Excel


# Wczytanie pliku Excel
dane_gieldowe = pd.read_excel('Market_data.xlsx')

strategies =pd.read_excel('Strategies_data.xlsx')
# 
# print(dane_gieldowe.head())




folder_data_input='row_data'
folder_output='output_simulations'

#strategy='0.4'
intervals=[ '2h', '1h', '30m', '15m'] # '30m', '15m' '1w', '3d','2d', '1d', '12h', '6h', '4h',
#intervals=['4h']




os.makedirs(folder_output ,exist_ok=True)
for interval in intervals:
    for idx, row in strategies.iterrows():
        strategy = row['Numer']
        # description = row['Wskaźniki']
        #print(f"Strategia {name}: Wskaźniki → {description}")


        for index, row in dane_gieldowe.iterrows():
            ticker = row['Ticker']
            ticker_plik = row['Plik']
            output_path_trades=f'{folder_output}/' + ticker + '- strategy ' + str(strategy) + ', interval ' + interval + ' - trades .xlsx'
            output_path_history=f'{folder_output}/' + ticker + '- strategy ' + str(strategy) + ', interval ' + interval + ' - history .xlsx'
            if os.path.exists(output_path_trades) and os.path.exists(output_path_history)  : # If is data for simulation, skip processing simulation
                #if(str(strategy) != "0.3"): # Temporary fixing strategies for 3 
                continue
            ticker = row['Ticker']
            ticker_plik = row['Plik']
            start_date=pd.to_datetime(row['Początek'])
            end_date=pd.to_datetime(row['Koniec'])
            print(f"interval: {interval} , strategy: {strategy}")
            data=get_local_data(ticker_plik, interval, folder_data_input)
            #print(interval)
            data['timestamp'] = pd.to_datetime(data['timestamp'], format="%Y-%m-%d")
            #print(data.head())
            #print(start_date)
            strategy=str(strategy)
            interval_dict = {}
            using_intervals=[interval]
            interval_obj = Interval(ticker, interval, data )
            interval_dict[using_intervals[-1]] = interval_obj
            
            data_proc=process_data(data)
            trades, history =backtest(interval_dict, ticker, strategy, using_intervals,  start_date, end_date , start_balance=10000, position_risk=0.002, stop_loss=False)
            history.to_excel(output_path_history, index=False)
            trades.to_excel(output_path_trades, index=False)
            





interval: 2h , strategy: 0.2


C:\Users\Arek\AppData\Local\Temp\ipykernel_32244\195449821.py:3: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  return pd.concat([account_history, new_row], ignore_index=True)


interval: 2h , strategy: 0.2


C:\Users\Arek\AppData\Local\Temp\ipykernel_32244\694806404.py:311: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '138702.51' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  data.loc[i, 'OBV'] = data.loc[i - 1, 'OBV'] + data.loc[i, 'volume']
C:\Users\Arek\AppData\Local\Temp\ipykernel_32244\195449821.py:3: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  return pd.concat([account_history, new_row], ignore_index=True)


interval: 2h , strategy: 0.2


...

C:\Users\Arek\AppData\Local\Temp\ipykernel_32244\694806404.py:313: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '-35652.508' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  data.loc[i, 'OBV'] = data.loc[i - 1, 'OBV'] - data.loc[i, 'volume']
C:\Users\Arek\AppData\Local\Temp\ipykernel_32244\195449821.py:3: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  return pd.concat([account_history, new_row], ignore_index=True)


interval: 2h , strategy: 0.2


C:\Users\Arek\AppData\Local\Temp\ipykernel_32244\195449821.py:3: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  return pd.concat([account_history, new_row], ignore_index=True)


interval: 2h , strategy: 0.2


C:\Users\Arek\AppData\Local\Temp\ipykernel_32244\694806404.py:313: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '-10549.856' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  data.loc[i, 'OBV'] = data.loc[i - 1, 'OBV'] - data.loc[i, 'volume']
C:\Users\Arek\AppData\Local\Temp\ipykernel_32244\195449821.py:3: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  return pd.concat([account_history, new_row], ignore_index=True)
